## 2 Layer MLP
- With hidden layer of size 64 and dropout with 0.5 probability
- Softmax at last classification layer 
- Learning rate of 0.001
- Over 50-100 epochs

In [4]:
import numpy as np
import tensorflow as tf
import argparse
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error, root_mean_squared_error, mean_absolute_error
import math
import wandb

from wandb.integration.keras.callbacks import WandbMetricsLogger, WandbModelCheckpoint


encoder = load_model("../Models/ae_encoder_mixed.keras")
X = np.load("../Data/LPS/sal-lps-150-1500(labeled)_data.npy")
Y = np.load("../Data/LPS/sal-lps-150-1500(labeled)_labels.npy")
print(X.shape)


(3274, 67499)


In [8]:
wandb.init(
    project="MLPClassifier",
    config={
        "hidden_size": 64,
        "activation": "relu",
        "dropout": 0.5,
        "output_activation": "softmax",
        "optimizer": "adam",
        "loss": "sparse_categorical_crossentropy",
        "metric": "accuracy",
        "epochs": 50,
        "batch_size": 32,
        "learning_rate": 0.001
    }
)

config = wandb.config


wandb: Currently logged in as: trapezium (trapezium-wits-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [9]:
class MLPClassifier(tf.keras.Model):
    def __init__(self, encoder, num_classes=2, hidden_size=64, dropout_rate=0.5):
        super().__init__()
        self.encoder = encoder
        self.encoder.trainable = False  # Freeze encoder
        self.hidden = layers.Dense(hidden_size, activation='relu')
        self.dropout = layers.Dropout(dropout_rate)
        self.classifier = layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        x = self.encoder(inputs)
        x = self.hidden(x)
        x = self.dropout(x)
        return self.classifier(x)

In [10]:
# Split into train (70%), val (10%) and test (20%) 
X_train, X_temp, y_train, y_temp = train_test_split(X, Y, test_size=0.3, random_state=42, stratify=Y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=2/3, random_state=42, stratify=y_temp)

In [11]:
print(f'Training data shape: {X_train.shape}')
print(f'Val data shape: {X_val.shape}')
print(f'Test data shape: {X_test.shape}')


Training data shape: (2291, 67499)
Val data shape: (327, 67499)
Test data shape: (656, 67499)


In [ ]:
model = MLPClassifier(encoder,hidden_size=config.hidden_size)

early_stopping = EarlyStopping(
    monitor='accuracy',
    patience=3,
    restore_best_weights=True
)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'],
            )

print("Training started....")
history = model.fit(
    X_train, y_train,
    epochs=50, batch_size=32,    
    validation_data=(X_val, y_val), 
    verbose=1,
    callbacks=[early_stopping,
               WandbMetricsLogger(),
               WandbModelCheckpoint("models.keras",save_best_only=True,save_weights_only=True,monitor='accuracy')
               ]
)
wandb.finish()
print("Training completed!")


Training started....
Epoch 1/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 53s 714ms/step - accuracy: 0.6111 - loss: 0.6777 - val_accuracy: 0.7034 - val_loss: 0.6525
Epoch 2/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 63s 879ms/step - accuracy: 0.6770 - loss: 0.6386 - val_accuracy: 0.6972 - val_loss: 0.6060
Epoch 3/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 50s 700ms/step - accuracy: 0.7289 - loss: 0.5900 - val_accuracy: 0.7645 - val_loss: 0.5574
Epoch 4/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 39s 552ms/step - accuracy: 0.7381 - loss: 0.5487 - val_accuracy: 0.7829 - val_loss: 0.5227
Epoch 5/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 52s 731ms/step - accuracy: 0.7857 - loss: 0.5094 - val_accuracy: 0.8135 - val_loss: 0.4944
Epoch 6/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 40s 555ms/step - accuracy: 0.7914 - loss: 0.4827 - val_accuracy: 0.8073 - val_loss: 0.4707
Epoch 7/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 42s 590ms/step - accuracy: 0.8079 - loss: 0.4544 - val_accuracy: 0.8226 - val_loss: 0.4622
Epoch 8/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 36s 499ms/step - accuracy: 0.8228 - lo

In [ ]:
model.summary()

Model: "mlp_classifier"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ (None, 500)            │   343,001,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │        32,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 343,098,084 (1.28 GB)

 Trainable params: 32,194 (125.76 KB)

 Non-trainable params: 343,001,500 (1.28 GB)

 Optimizer params: 64,390 (251.53 KB)

In [ ]:
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, ConfusionMatrixDisplay

loss, accuracy = model.evaluate(X_test, y_test)

y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)


cm = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cm.ravel()
print(f'TN: {tn}, FP: {fp}, FN: {fn}, TP: {tp}')

if len(np.unique(y_test)) == 2:
    auc = roc_auc_score(y_test, y_pred_probs[:, 1])
    print(f'AUC: {auc}')
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_probs[:, 1])
    import matplotlib.pyplot as plt
    plt.plot(fpr, tpr, label=f'AUC = {auc:.2f}')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend()
    plt.show()

# Confusion matrix plot
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()
print(f'Test loss: {loss}')
print(f'Test Accuracy {accuracy}')

21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 104ms/step - accuracy: 0.8780 - loss: 0.2965
Test loss: 0.2964734733104706
Test Accuracy 0.8780487775802612


In [ ]:
# spectrum = 50
# test = model.predict(X_test[:spectrum])
# print(np.argmax(test,axis=1))
# print(y_test[:spectrum])
# # print(np.array([X_test[0]]).shape)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
[0 0 0 1 1 1 1 0 1 0 1 1 0 1 0 0 1 1 0 1 0 1 0 1 0 0 1 1 1 1 1 1 1 0 0 1 0
 1 0 1 0 1 0 1 1 0 0 0 1 1]
[0 0 0 1 1 1 1 0 1 0 1 0 0 1 0 0 1 1 0 1 0 1 0 1 0 0 1 1 1 0 1 1 1 0 1 1 0
 1 0 1 0 1 0 1 1 0 0 0 1 1]


In [ ]:
# model.save(filepath="../Models/Pretrained_AE_Classifier.keras")